# Project 2

Name: Dylan Connor  
Andrew ID: dmconnor

## KML Workbook

In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
import seaborn as sns
import math
import datetime

In [3]:
#Lat, Lon and status data
data_X = pd.read_csv("output_data/ML_Final_Data_v2.csv", usecols = ['status', 'lat','lon'])
data_X = data_X.dropna()

In [16]:
#Group by lat lon and status to get aggregate counts
to_plot = data_X.groupby(['lat','lon','status']).apply(lambda df : len(df)).unstack().reset_index()

In [17]:
to_plot['funded'].value_counts().head()

1.0    34
5.0    13
2.0    13
4.0    10
3.0     8
Name: funded, dtype: int64

In [18]:
to_plot_exp = to_plot.copy()
to_plot_fund = to_plot.copy()
to_plot_exp.drop(to_plot_exp.index, inplace=True)
to_plot_fund.drop(to_plot_fund.index, inplace=True)
to_plot_fund.shape

(0, 4)

In [19]:
for index, row in to_plot.iterrows():
    if not math.isnan(row['funded']):        
        a = int(row['funded'])
        while a > 1000 :
            to_plot_fund.loc[len(to_plot_fund)] = row
            a = a - 1000

In [20]:
to_plot_fund.shape

(438, 4)

In [21]:
for index, row in to_plot.iterrows():
    if not math.isnan(row['expired']):        
        a = int(row['expired'])
        while a > 50 :
            to_plot_exp.loc[len(to_plot_exp)] = row
            a = a - 50

In [22]:
to_plot_exp.shape

(661, 4)

In [23]:
#data_X = pd.read_csv("output_data/ML_Final_Data.csv", usecols = ['status','lat','lon'])
lat = data_X['lat'].values
lon = data_X['lon'].values
status = data_X['status'].values


In [27]:
#Create KML string 
kml_header = ("<?xml version=\"1.0\" encoding=\"UTF-8\" ?>\n" 
        + "<kml xmlns=\"http://earth.google.com/kml/2.2\">\n"
        + "<Document>\n"
        + "\t<Style id=\"style1\">\n"
        + "\t\t<IconStyle>\n"
        + "\t\t\t<Icon>\n"
        + "\t\t\t\t<href>https://maps.google.com/mapfiles/kml/pal3/icon20.png\n</href>"
        + "\t\t\t</Icon>\n"
        + "\t\t</IconStyle>\n"
        + "\t</Style>\n"
        + "\t<Style id=\"style2\">\n"
        + "\t\t<IconStyle>\n"
        + "\t\t\t<Icon>\n"
        + "\t\t\t\t<href>https://maps.google.com/mapfiles/kml/pal3/icon46.png\n</href>"
        + "\t\t\t</Icon>\n"
        + "\t\t</IconStyle>\n"
        + "\t</Style>\n")
kml_footer = "</Document>\n</kml>"

kml_coord = ""

# Add funded
for index, row in to_plot_fund.iterrows():
    kml_coord = (kml_coord + "<Placemark>\n"
                + "\t<styleUrl>#style1</styleUrl>\n"
                + "\t<Point>\n"
                + "\t\t<coordinates>" + str(row['lon']) + "," + str(row['lat']) + "</coordinates>\n"
                + "\t</Point>"
                + "</Placemark>\n")

# Add expired
for index, row in to_plot_exp.iterrows():
        kml_coord = (kml_coord + "<Placemark>\n"
                + "\t<styleUrl>#style2</styleUrl>\n"
                + "\t<Point>\n"
                + "\t\t<coordinates>" + str(row['lon']) + "," + str(row['lat']) + "</coordinates>\n"
                + "\t</Point>"
                + "</Placemark>\n")
    

In [28]:
#Write to kml
kml = kml_header + kml_coord + kml_footer

f = open('kiva2.kml','w')
f.write(kml)
f.close()
